# Crawler for ATM location Data

In [23]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import csv
import pandas

In [19]:
# Driver execute
driver = webdriver.Chrome('/Users/dangchan/Documents/chromedriver')
driver.get("https://map.naver.com/v5/search")
try:
   element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.XPATH, "/html/body/app/layout/dynamic-content-outlet/promotion-popup/div/button"))
   ) #입력창이 뜰 때까지 대기
finally:
   pass

# popup close
driver.find_element(by=By.XPATH, value= "/html/body/app/layout/dynamic-content-outlet/promotion-popup/div/button").click()

# search box
search_box = driver.find_element(by=By.CSS_SELECTOR, value="div.input_box>input.input_search")
search_box.send_keys("대전광역시 ATM")
time.sleep(2)
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# Iframe Switch
frame = driver.find_element(by=By.CSS_SELECTOR, value="iframe#searchIframe")

driver.switch_to.frame(frame)

/var/folders/6r/ndnspf4n6n7_m8rb9bs7qdvr0000gp/T/ipykernel_28086/4120535932.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/dangchan/Documents/chromedriver')


In [21]:
scroll_div = driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/div[1]")

# Scroll bar > 전체내용 표시되게끔
driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)

In [ ]:
# Crawling

result = []
